In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import pulp
from time import time

### Load and preprocess data

In [5]:
df = pd.read_excel('coffee coffee sales.xlsx')

def combine(row):
    
    return datetime.combine(row['Date'], row['Time'])

df['Datetime'] = df.apply(lambda x: combine(x), axis = 1)

# Group every 30mins
df = df.groupby(pd.Grouper(key = 'Datetime',freq='30Min')).sum()

# Sales to employee count
df['Employee Count'] = df['Sales']//110 +1
df = df[df['Sales'] != 0]
df.drop(columns = ['Sales'], inplace =True)
df.reset_index(inplace=True)

# Fill missing values
missing = pd.DataFrame([[datetime.strptime('2020-03-31 06:00:00', '%Y-%m-%d %H:%M:%S'), 1], [datetime.strptime('2020-04-04 07:00:00', '%Y-%m-%d %H:%M:%S'),1]], columns = ['Datetime', 'Employee Count'])
df = df.append(missing).sort_values(by = 'Datetime')
df.reset_index(drop = True, inplace = True)

# Convert time into a grid
time_unit_dict = dict(zip(df['Datetime'].apply(lambda x: x.time()).unique(), range(20)))
date_unit_dict = {29:0, 30:1, 31:2, 1:3, 2:4, 3:5, 4:6}

def finite_time(x):
    
    date_unit = date_unit_dict[x.day]
    time_unit = time_unit_dict[x.time()]
    
    return(date_unit, time_unit)

df['Unittime'] = df['Datetime'].apply(lambda x: finite_time(x))

### Constraint programming approach

In [3]:
# define parameters 
num_days = 7
num_time_units = 20
num_emps = 4
all_days = range(num_days)
all_time_units = range(num_time_units)
all_emps = range(num_emps)

reqs = dict(zip(df['Unittime'], df['Employee Count']))

# environment
cube = []
for i in range(num_days):
    for j in range(num_time_units):
        for k in range(num_emps):
            cube.append([i,j,k])

In [4]:
# Initialize model
prob = pulp.LpProblem('CreateStaffing', pulp.LpMinimize)

staffed = pulp.LpVariable.dicts("staffed",
                                   ((day, timeslot, staffmember) for day, timeslot, staffmember 
                                    in cube),
                                     lowBound=0,
                                     cat='Binary')

In [5]:
# Cost                             
prob += pulp.lpSum([staffed[day, timeslot, staffmember] for day, timeslot, staffmember in cube])

In [6]:
# Constraint 1: Meet demand
for day in all_days:
    for timeslot in all_time_units:
        prob += (sum([staffed[(day, timeslot, person)] for person in all_emps]) >= reqs[(day,timeslot)])

In [7]:
# Constraint 2: Shifts must be continuous
num_slots = max(all_time_units)
for day in all_days:
    for timeslot in all_time_units:
        if timeslot < num_slots:
            for person in all_emps:
                prob += staffed[day, timeslot+1, person] <= staffed[day, timeslot, person] + \
                    (1 - (1./num_slots) *
                     sum([staffed[(day, s, person)] for s in all_time_units if s < timeslot]))

In [8]:
# Constraint 3: Shift length min 3hrs max 8.5 hrs
for day in all_days:
    for person in all_emps:
        shift_length = sum(staffed[(day, timeslot, person)] for timeslot in all_time_units)
        prob += (shift_length <= 17)
        prob += (6 <= shift_length)

In [9]:
# Constraint 4: Staff Should get minimal hours a week. Requirement split amongst 4.
for person in all_emps:
    total =[]
    for day in all_days:
        for timeslot in all_time_units:
            total += staffed[(day, timeslot, person)]
    prob += (total >= sum(reqs.values())//num_emps)       

In [10]:
# solve
prob.solve()
print(pulp.LpStatus[prob.status])

Optimal


In [11]:
output = []
for day, timeslot, staffmember in staffed:
    var_output = {
        'Day': day,
        'Timeslot': timeslot,
        'Staffmember': staffmember,
        'Staffed': staffed[(day, timeslot, staffmember)].varValue,
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output)
output_df = output_df[output_df['Staffed'] != 0]

In [12]:
pulp.value(prob.objective)

247.0

In [13]:
output_df.to_csv('pulp_output.csv', index =False)

### Testing

In [17]:
for each in output_df.columns:
    output_df[each] = output_df[each].apply(int)

In [23]:
## Tests

# shifts should be consecutive
def time_slot_test(df):  
    for day in all_days:
        for person in all_emps:
            l = list(df[(df['Day']==day) & (df['Staffmember'] == person)]['Timeslot'])
            print(day,person)
            print(sorted(l) == list(range(min(l), max(l)+1)))

# Requirements are met            
def req_test(df):
    return False not in set(df.groupby(['Day', 'Timeslot']).sum()['Staffed'] >= np.array(list(reqs.values())))

# Shift length
def shift_length_test(df):
    for day in all_days:
        for person in all_emps:
            l = list(df[(df['Day']==day) & (df['Staffmember'] == person)]['Timeslot'])  
            print(6 <= len(l) <= 17)
        

In [25]:
time_slot_test(output_df)

0 0
True
0 1
True
0 2
True
0 3
True
1 0
True
1 1
True
1 2
True
1 3
True
2 0
True
2 1
True
2 2
True
2 3
True
3 0
True
3 1
True
3 2
True
3 3
True
4 0
True
4 1
True
4 2
True
4 3
True
5 0
True
5 1
True
5 2
True
5 3
True
6 0
True
6 1
True
6 2
True
6 3
True


In [38]:
output_df.to_csv('pulp_final.csv',index=False)

### Convert to required format

In [6]:
output_df =  pd.read_csv('pulp_final.csv')
output_df['Date'] = output_df.iloc[:,[0,1]].values.tolist()
output_df['Date'] = output_df['Date'].apply(tuple)
output_df.drop(columns = ['Day', 'Timeslot', 'Staffed'], inplace = True)

output_df = pd.merge(output_df, df, left_on='Date', right_on='Unittime')
output_df.drop(columns =['Date', 'Employee Count', 'Unittime'], inplace =True)
output_df.set_index('Datetime', inplace = True)

In [7]:
grouped = output_df.groupby(by = [output_df.index.day, 'Staffmember'])

In [8]:
final_df = pd.DataFrame(columns = ['Date', 'Shift Start', 'Shift End'])
for k,g in grouped:
    final_df = pd.concat([final_df, pd.DataFrame({'Date':[g.index[0].date()], 'Shift Start':[g.index.min().time()], 'Shift End':[g.index.max().time()]})])

In [9]:
def add_delta(tme, delta):
    # transform to a full datetime first
    return (datetime.combine(date.today(), tme) + 
            delta).time()

In [10]:
final_df['Shift End']= final_df['Shift End'].apply(lambda x: add_delta(x, timedelta(minutes=30)))

In [11]:
def diff_time(row):
    x = row['Shift Start']
    y = row['Shift End']
    diff = datetime.combine(date.today(), y) - datetime.combine(date.today(), x)
    return diff.total_seconds() / (3600)   

In [12]:
final_df['delta'] = final_df.apply(lambda x: diff_time(x), axis = 1)

In [15]:
final_df.sort_values(['Date', 'Shift Start'], inplace = True)

In [17]:
final_df.to_csv('final.csv', index =False)